# 도표 8-6. ChatGPT에게 수요예측 정확도 1차 산출 요청

전체 수요예측정확도를 계산한다.
먼저 과거 주차별로 수요예측 정확도를 계산한다. 그리고 과거 주차별 가중치를 적용한다.
과거 특정 주차의 수요에측 정확도는 특정 주차의 모든 모델에 대한 예측치와 당주 실적의 비교를 통해 계산한다.
해당 모델에 대해서 1주 전주부터 8주 전까지 아래와 같이 당주에 대한 에측치와 당주 실적을 비교하여 모델_과거주차_MIM, 모델_과거주차_MAX에 저장한다.
모델_과거주차_MIN은 과거 주차의 당주 예측치와 당주 실적 중 작은 값이다.
모델_과거주차_MAX는과거 주차의 당주 예측치와 당주 실적 중 크거나 같은 값이다.
예를 들어, 모델01에 대해  8주 전에 당주에 대한 에측치와 당주 실적을 비교하여 작은 값을 모델01_8_MIN에, 크거나 같은 값은 모델01_8_MAX에 저장한다.
모델02에 대해  7주 전에 당주에 대한 에측치와 당주 실적을 비교하여 작은 값을 모델02_7_MIN에, 크거나 같은 값은 모델02_7_MAX에 저장한다.
모든 과거주차에 대해서 수요예측 정확도를 계산한다.
모든 모델에 대해 과거주차_MIN을 합산하여 T_과거주차_MIN에 저장한다.
예를 들어 8주 전의 T_8_MIM은 모델01_8_MIN + 모델02_8_MIN  + ...+ 모델03_8_MIN이다.
예를 들어 7주 전의 T_7_MIM은 모델01_7_MIN + 모델02_7_MIN  + ...+ 모델03_7_MIN이다.
예를 들어 6주 전의 T_6_MAX는 모델01_6_MAX + 모델02_6_MAX  + ...+ 모델03_6_MAX이다.
예를 들어 5주 전의 T_5_MAX는 모델01_5_MAX + 모델02_5_MAX  + ...+ 모델03_5_MAX이다.
과거주차의 수요예측 정확도는 T_과거주자_MIM / T_ 과거주차 MAX 이다.
예를 들어, 8주전의 수요에측정확도는  T_8_MIN/T_8_MAX 이다.
7주전의 수요에측정확도는  T_7_MIN/T_7_MAX 이다
전체 수요에측정확도는 과거 모든 주차의 수요예측정확도의 가중 평균이다.
과거 주차의 가중치는 8주전 5%, 7주전 5%, 6주전 10%, 5주전 10%, 4주전 15%, 3주전 15%, 2주전 20%, 1주전 20%이다.
한글로 설명해줘

In [1]:
import pandas as pd

# Load data from both files
forecast_df = pd.read_excel('/content/sample_data/MODEL_FORECAST.xlsx')
actual_df = pd.read_excel('/content/sample_data/MODEL_ACTUAL.xlsx')

# Display the first few rows of each dataframe
forecast_df.head(), actual_df.head()


(     모델 과거주차  에측치
 0  모델01  8주전  100
 1  모델01  7주전  150
 2  모델01  6주전   50
 3  모델01  5주전  120
 4  모델01  4주전   80,
      모델  주차   실적
 0  모델01  당주  100
 1  모델02  당주  200
 2  모델03  당주  300)

In [2]:
# Mapping actual sales to each model and week from forecast data
# Assuming all actual sales data is for the same week "당주", and need to map it to all forecast weeks
merged_df = forecast_df.copy()
actual_dict = actual_df.set_index('모델')['실적'].to_dict()
merged_df['실적'] = merged_df['모델'].map(actual_dict)

# Calculate MIN and MAX
merged_df['MIN'] = merged_df.apply(lambda row: min(row['에측치'], row['실적']), axis=1)
merged_df['MAX'] = merged_df.apply(lambda row: max(row['에측치'], row['실적']), axis=1)

merged_df.head()


,모델,과거주차,에측치,실적,MIN,MAX
0,모델01,8주전,100,100,100,100
1,모델01,7주전,150,100,100,150
2,모델01,6주전,50,100,50,100
3,모델01,5주전,120,100,100,120
4,모델01,4주전,80,100,80,100


In [3]:
# Summing MIN and MAX values for each past week
summary_df = merged_df.groupby('과거주차').agg({'MIN': 'sum', 'MAX': 'sum'}).reset_index()

# Calculating accuracy for each week
summary_df['Accuracy'] = summary_df['MIN'] / summary_df['MAX']
summary_df


,과거주차,MIN,MAX,Accuracy
0,1주전,300,600,0.500000
1,2주전,270,600,0.450000
2,3주전,320,610,0.524590
3,4주전,240,600,0.400000
4,5주전,340,620,0.548387
5,6주전,150,600,0.250000
6,7주전,400,650,0.615385
7,8주전,300,600,0.500000


In [4]:
# Mapping weights to each week
weights = {
    '8주전': 0.05,
    '7주전': 0.05,
    '6주전': 0.10,
    '5주전': 0.10,
    '4주전': 0.15,
    '3주전': 0.15,
    '2주전': 0.20,
    '1주전': 0.20
}

# Applying weights
summary_df['Weight'] = summary_df['과거주차'].map(weights)
summary_df['Weighted Accuracy'] = summary_df['Accuracy'] * summary_df['Weight']

# Calculate the weighted average accuracy
total_accuracy = summary_df['Weighted Accuracy'].sum() / summary_df['Weight'].sum()
total_accuracy


0.4642964650368141

# 도표 8-9. 수정한 수요예측 정확도 산출 요청 내용

당주(9주차)에 대한 전체 모델의 수요 예측 정확도를 계산하려고 합니다. 과거 1주에서 8주까지 각 모델의 주차별 예측치와 실적을 비교하여, 다음과 같은 절차로 계산을 진행해 주세요:
데이터 확인: 각 모델의 1주차부터 8주차까지의 예측치와 9주차의 실적 데이터를 확인합니다.
최소값과 최대값 계산: 각 주차와 모델별로 예측치와 실적을 비교하여 최소값과 최대값을 구합니다.
주차별 합계: 각 주차별로 모든 모델의 최소값과 최대값의 합을 구합니다.
수요 예측 정확도 계산: 각 주차별로 최소값 합계를 최대값 합계로 나누어 정확도를 계산합니다.
가중 평균 계산: 8주 전 5%, 7주 전 5%, 6주 전 10%, 5주 전 10%, 4주 전 15%, 3주 전 15%, 2주 전 20%, 1주 전 20%의 가중치를 적용하여 전체 수요 예측 정확도를 계산합니다.

# 도표 8-10. 보다 현실적인 예측, 실적정보의 형태 파일 업로드 파이썬 코드

In [5]:
import pandas as pd

# Load the provided Excel files
actual_path = '/content/sample_data/ACTUAL_당주.xlsx'
forecast_path = '/content/sample_data/FORECAST_주차.xlsx'

# Read the data from the Excel files
actual_data = pd.read_excel(actual_path)
forecast_data = pd.read_excel(forecast_path)

# Display the first few rows of each dataset to understand their structure
actual_data.head(), forecast_data.head()


(     모델  주차   실적
 0  모델01   9  100
 1  모델02   9  200
 2  모델03   9  300,
      모델  주차  예측치
 0  모델01   1  100
 1  모델01   2  150
 2  모델01   3   50
 3  모델01   4  120
 4  모델01   5   80)

# 도표 8-11. 모델별 주차별 최소값. 최대값 계산 파이썬 코드

In [6]:
# 예측 데이터와 실적 데이터를 '모델' 컬럼을 기준으로 결합합니다.
# 이를 통해 각 모델의 예측치와 실적이 같은 행에 위치하게 됩니다.
merged_data = forecast_data.merge(actual_data, on='모델', suffixes=('_예측', '_실적'))

# 'apply' 함수를 사용하여 각 행에 대해 최소값과 최대값을 계산합니다.
# 여기서 'min' 함수는 예측치와 실적 중 더 작은 값을 선택하고,
# 'max' 함수는 더 큰 값을 선택합니다.
merged_data['최소값'] = merged_data.apply(lambda row: min(row['예측치'], row['실적']), axis=1)
merged_data['최대값'] = merged_data.apply(lambda row: max(row['예측치'], row['실적']), axis=1)

merged_data.head()

,모델,주차_예측,예측치,주차_실적,실적,최소값,최대값
0,모델01,1,100,9,100,100,100
1,모델01,2,150,9,100,100,150
2,모델01,3,50,9,100,50,100
3,모델01,4,120,9,100,100,120
4,모델01,5,80,9,100,80,100


# 도표 8-12. 주차별 예측결과와 전체 수요예측 정확도 계산 파이썬 코드

In [7]:
# 주차별로 그룹화(groupby)한 후, 각 주차의 최소값과 최대값의 합을 계산합니다.
# 이렇게 하면 각 주차별로 예측치와 실적 중 더 작은 값과 더 큰 값들의 합을 얻을 수 있습니다.
weekly_sums = merged_data.groupby('주차_예측').agg({'최소값': 'sum', '최대값': 'sum'}).reset_index()

# 각 주차별로 정확도를 계산합니다. 정확도는 주차별 최소값의 합을 최대값의 합으로 나눈 값입니다.
# 이는 예측치가 실적에 얼마나 근접했는지를 비율로 나타냅니다.
weekly_sums['정확도'] = weekly_sums['최소값'] / weekly_sums['최대값']

# 각 주차별로 정확도에 적용할 가중치를 설정합니다.
# 이는 과거 데이터의 시간적 중요도를 반영하여 최근 주차에 더 높은 가중치를 부여합니다.
weights = {1: 0.05, 2: 0.05, 3: 0.10, 4: 0.10, 5: 0.15, 6: 0.15, 7: 0.20, 8: 0.20}

# 가중 평균 정확도를 계산합니다. 이는 각 주차의 정확도에 해당 주차의 가중치를 곱한 후,
# 모든 주차의 결과를 합하여 총 가중 평균을 구합니다.
weighted_accuracy = (weekly_sums['정확도'] * weekly_sums['주차_예측'].map(weights)).sum()

# 주차별 계산된 최소값, 최대값, 정확도와 전체 가중 평균 정확도를 확인할 수 있습니다.
weekly_sums, weighted_accuracy

(   주차_예측  최소값  최대값       정확도
 0      1  300  600  0.500000
 1      2  400  650  0.615385
 2      3  150  600  0.250000
 3      4  340  620  0.548387
 4      5  240  600  0.400000
 5      6  320  610  0.524590
 6      7  270  600  0.450000
 7      8  300  600  0.500000,
 0.4642964650368141)

In [8]:
# 각 주차별로 정확도에 적용할 가중치를 리스트로 설정합니다.
weights = [0.05, 0.05, 0.10, 0.10, 0.15, 0.15, 0.20, 0.20]

# 가중 평균 정확도를 계산합니다. 이는 각 주차의 정확도에 해당 주차의 인덱스에 맞는 가중치를 곱한 후,
# 모든 주차의 결과를 합하여 총 가중 평균을 구합니다.
# 주차_예측 값을 인덱스로 사용하기 위해 1을 빼줍니다 (인덱스 0부터 시작).
weighted_accuracy = (weekly_sums['정확도'] * weekly_sums['주차_예측'].apply(lambda x: weights[x-1])).sum()

# 주차별 계산된 최소값, 최대값, 정확도와 전체 가중 평균 정확도를 확인할 수 있습니다.
weekly_sums, weighted_accuracy


(   주차_예측  최소값  최대값       정확도
 0      1  300  600  0.500000
 1      2  400  650  0.615385
 2      3  150  600  0.250000
 3      4  340  620  0.548387
 4      5  240  600  0.400000
 5      6  320  610  0.524590
 6      7  270  600  0.450000
 7      8  300  600  0.500000,
 0.4642964650368141)